In [23]:
#install kaggle
!pip install kaggle


Defaulting to user installation because normal site-packages is not writeable


In [24]:
import sys
print(sys.executable)


C:\Users\ejazc\AppData\Local\Programs\Python\Python313\python.exe


In [25]:
!"C:\Users\ejazc\AppData\Local\Programs\Python\Python313\python.exe" -m pip install --upgrade pip

In [26]:
import kaggle

In [27]:
!pip show kaggle

Name: kaggle
Version: 1.7.4.5
Summary: Access Kaggle resources anywhere
Home-page: https://github.com/Kaggle/kaggle-api
Author: 
Author-email: Kaggle <support@kaggle.com>
License: Apache License
                           Version 2.0, January 2004
                        http://www.apache.org/licenses/

   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION

   1. Definitions.

      "License" shall mean the terms and conditions for use, reproduction,
      and distribution as defined by Sections 1 through 9 of this document.

      "Licensor" shall mean the copyright owner or entity authorized by
      the copyright owner that is granting the License.

      "Legal Entity" shall mean the union of the acting entity and all
      other entities that control, are controlled by, or are under common
      control with that entity. For the purposes of this definition,
      "control" means (i) the power, direct or indirect, to cause the
      direction or management of such entity,

In [28]:
!where kaggle

C:\Users\ejazc\AppData\Local\Programs\Python\Python313\Scripts\kaggle.exe


In [29]:
import os
os.environ['PATH'] += r";C:\Users\ejazc\AppData\Local\Programs\Python\Python313\Scripts"

In [30]:
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0
orders.csv: Skipping, found more recently modified local copy (use --force to force download)


In [31]:
#import pandas
import pandas as pd

In [32]:
# read the dataset and removed 'Not Available', 'unknown' with 'NAN'
df = pd.read_csv('orders.csv',na_values=['Not Available', 'unknown'])
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [33]:
#convert the column name to lower case and removes=d white spaces with "_"
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ','_')
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'cost_price', 'list_price', 'quantity',
       'discount_percent'],
      dtype='object')

In [34]:
#derive new column discount , sale price and profit
df["discount"] = df["list_price"]*df["discount_percent"]*.01
df["sale_price"]= df["list_price"]-df["discount"]
df["profit"] = df["sale_price"] - df["cost_price"]

In [35]:
#convert order date from obj to datetime
df["order_date"]=pd.to_datetime(df["order_date"],format="%Y-%m-%d")

In [36]:
#drop columns - list price , discount percent & order
df.drop(columns=['list_price','discount_percent','cost_price'],inplace=True)

In [37]:
df.head(2)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,quantity,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,3,21.9,708.1,108.1


In [38]:
!pip install pymysql sqlalchemy

Defaulting to user installation because normal site-packages is not writeable


In [39]:

import urllib.parse
from sqlalchemy import create_engine, text

import pymysql
import pandas as pd


In [40]:
PASSWORD = 'Ejaz@chaudhary123'
encoded_password = urllib.parse.quote_plus(PASSWORD)

In [41]:
USER = 'root'                # MySQL username
PASSWORD = urllib.parse.quote_plus('Ejaz@chaudhary123')
HOST = '127.0.0.1'           # keep as localhost
PORT = '3306'                # default MySQL port
DB_NAME = 'retail_order'  # name of your database


In [42]:
connection_string = f"mysql+pymysql://{USER}:{encoded_password}@{HOST}:{PORT}/{DB_NAME}"
engine = create_engine(connection_string)

try:
    with engine.connect() as conn:
        result = conn.execute(text("SELECT DATABASE();"))
        print("✅ Connected to database:", result.scalar())
except Exception as e:
    print("❌ Connection failed:", e)


✅ Connected to database: retail_order


In [43]:
df.to_sql('cleaned_retail_order', con=engine, if_exists='append', index=False)
print("✅ Cleaned data uploaded successfully to MySQL as 'orders_cleaned' table!")

✅ Cleaned data uploaded successfully to MySQL as 'orders_cleaned' table!
